In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def merge_excel(path):  #输入路径，得到合并后的数据
    data = pd.DataFrame()
    #data.info()
    #temp_data = pd.DataFrame()
    files = os.listdir(path)
    for i,file in enumerate (files):
        file_path = os.path.join(path,file)
        temp_data = pd.read_excel(file_path,names=None,sheet_name='Sheet1',index_col=None,header=None) #读取excel
        #print(data.shape);print(temp_data.shape)
        data = pd.concat([data,temp_data],axis=0)  #按行合并
    #data.info()
    print(path,"的合并数据大小为： ",data.shape)
    return data

def merge_data(df1,df2):
    merge_df=pd.concat([df1,df2],axis=0)
    print("合并后的有效数据大小为： ",merge_df.shape)
    return merge_df

def process_data(df):
    data=df

    data.drop_duplicates(inplace =True)   #删除重复行数据
    data.rename(columns ={0:'data',1:'ID',2:'name',3:'gender',4:'age',5:'position',
                              6:'Acquisition',7:'registration',8:'x',9:'y',10:'z',11:'comment',12:'other'},inplace =True)
    #data.drop(['registration','data','comment','other'],axis=1,inplace=True)   #删除无用列，不确定拿到后续数据的扩展性，弃用此方法
    #data = data.set_index('data')
    data = data.loc[:,['ID','name','gender','age','position','Acquisition','x','y','z']] #保留想要的列
    info_col = ['ID','name','gender','age','position','Acquisition'] #需补全信息列
    dat_col = ['x','y','z']   #数据列
    data[info_col] = data[info_col].ffill()   #info列填充上一行的非空值，赋值操作，不需要inplace 操作
    
    data = data.groupby('ID').filter(lambda x: len(x)!=1)  #筛选掉所有只有单次的数据
    
    #把x,y,z三列转换为数据类型，不能转成的转为nan，写的文字和角度都被转为999,用作下一步筛选  
    data[dat_col] = data[dat_col].apply(pd.to_numeric, errors='coerce').fillna(999)
    data.loc[(abs(data.x)>0.55)|(abs(data.y)>0.55)|(abs(data.z)>0.55)] = np.nan   #误差＞0.55都重复摆位，转为NaN数据
    data.dropna(inplace = True)                                                   #删除有空值的行，此行数据均为摆位超标数据
    
    #data.info()
    #print("处理后的有效数据大小为： ",data.shape)
    return data

def cal_mean_std(df):
    cal_data = df
    mean_num = [] ; std_num = []
    x_mean = cal_data.groupby(['ID'])['x'].mean().tolist()        #提取x方向的摆位误差均值，按照ID区分不同病人数据,并转为成列表
    mean_num.append(x_mean)
    x_std  = cal_data.groupby(['ID'])['x'].std(ddof=0).tolist()  #提取x方向的摆位误差标准差，ddof=0在pandas中表示求样本标准差
    std_num.append(x_std)
    y_mean = cal_data.groupby(['ID'])['y'].mean().tolist()
    mean_num.append(y_mean)
    y_std = cal_data.groupby(['ID'])['y'].std(ddof=0).tolist()
    std_num.append(y_std)
    z_mean = cal_data.groupby(['ID'])['z'].mean().tolist()
    mean_num.append(z_mean)
    z_std = cal_data.groupby(['ID'])['z'].std(ddof=0).tolist()
    std_num.append(z_std)
    #mean_num 和std_num 长度均为3，包括3个列表，代表X,Y,Z三个方向的数据，其中每个列表的个数代表由多少个不同的ID号也即是病例数
    if len(mean_num[1])<=1 or len(std_num[1])<=1:
        print("数据不足计算！")
        
    print("患者病例数：",len(mean_num[1]))
    #print(mean_num,std_num)
    return mean_num,std_num

def cal_sigma_delta(mean,std):
    sigma = [];delta = []
    for i in range (3):
        sigma.append(np.std(mean[i],ddof = 1));
        delta.append(np.sqrt(np.mean(np.array(std[i])**2)))        
    return sigma,delta

def cal_margin(sigma,delta):
    sigma = 10*np.array(sigma); delta = 10*np.array(delta)
    extend_margin = 2.5*sigma+0.7*np.array(delta)
    print("x方向推荐外放边界为：",round(extend_margin[0],3),"mm")
    print("y方向推荐外放边界为：",round(extend_margin[1],3),"mm")
    print("z方向推荐外放边界为：",round(extend_margin[2],3),"mm")
    return extend_margin

def screen_data(data):
    #data_head = data.groupby('position').filter(lambda x: str(x)=='头部')
    #data_chest = data.groupby('position').filter(lambda x: str(x)=='胸部')
    #data_abdomen = data.groupby('position').filter(lambda x: str(x)=='腹部')
    #data_head=data['position'].str.startswith('头')
    data_head = data.loc[data['position'].str.contains('头|颈|脑|鼻|喉|舌|咽|颌|唇|口|耳|腭|扁桃|腮腺|颞叶|颊|眼|NK|鄂|颅|甲状腺|牙龈|声带|肩|锁骨|枕叶|面部')]
    data_chest = data.loc[data['position'].str.contains('胸|纵隔|肺|食管|乳|腋|肋|支气管|胸部|ESO')]
    data_abdomen = data.loc[data['position'].str.contains('肝|肾|胃|贲门|结肠|小肠|腹')]
    data_pelvis = data.loc[data['position'].str.contains('宫颈|前列腺|直肠|卵巢|精原|髂骨|子宫|盆腔|睾丸|肛|股骨|膀胱|阴茎|阴道|髋')]
    
    #data.info()
    #data_head.info()
    return data_head,data_chest,data_abdomen,data_pelvis

def manual_screen_data(data,params):  #如果输入*params的话 ，就是把输入参数再作为一个元组
    print(params)
    data_screen = pd.DataFrame()
    for param in params:
        print(param)
        data_temp = data.loc[data['position'].str.contains(str(param))]
        data_screen = pd.concat([data_screen,data_temp],axis=0)
    return data_screen

In [ ]:
NO_1_path = 'E:\paper\摆位误差分析\一号机CBCT'; NO_3_path ='E:\paper\摆位误差分析\三号机CBCT'
data_1 = merge_excel(NO_1_path)  #合并文件内表格数据，并转为dataframe

data_3 = merge_excel(NO_3_path)
data_origin=merge_data(data_1,data_3)
data_1=process_data(data_1)    #数据预处理

data_3=process_data(data_3)

final_data = merge_data(data_1,data_3)
final_data = process_data(final_data)
print("完整数据大小为： ",final_data.shape)
data_head,data_chest,data_abdomen,data_pelvis = screen_data(final_data)
#print(data.head(10))
print("头颈部肿瘤数据:")
print("数据大小为：",data_head.shape)
mean_num,std_num = cal_mean_std(data_head)    #计算不同患者的摆位误差的平均值和标准差
sigma,delta = cal_sigma_delta(mean_num,std_num)              #计算sigma和delta
cal_margin(sigma,delta);                                              #计算外放边界

print("胸部肿瘤数据:")
print("数据大小为：",data_chest.shape)
mean_num,std_num = cal_mean_std(data_chest)    #计算不同患者的摆位误差的平均值和标准差
sigma,delta = cal_sigma_delta(mean_num,std_num)              #计算sigma和delta
cal_margin(sigma,delta);                                              #计算外放边界

print("腹部肿瘤数据:")
print("数据大小为：",data_abdomen.shape)
mean_num,std_num = cal_mean_std(data_abdomen)    #计算不同患者的摆位误差的平均值和标准差
sigma,delta = cal_sigma_delta(mean_num,std_num)              #计算sigma和delta
cal_margin(sigma,delta);                                              #计算外放边界

print("盆腔肿瘤数据:")
print("数据大小为：",data_pelvis.shape)
mean_num,std_num = cal_mean_std(data_pelvis)    #计算不同患者的摆位误差的平均值和标准差
sigma,delta = cal_sigma_delta(mean_num,std_num)              #计算sigma和delta
cal_margin(sigma,delta);                                              #计算外放边界



In [ ]:
cancer = ('乳腺','乳')#,'结肠'  #多个输入的时候整体输入为元组，单一输入的时候单一输入为变量，for循环会分别筛选“直”和“肠”字符

data_define = manual_screen_data(final_data,cancer);
print("%s数据为:%d条"%(cancer,len(data_define.index)))
mean_num,std_num = cal_mean_std(data_define)    #计算不同患者的摆位误差的平均值和标准差
sigma,delta = cal_sigma_delta(mean_num,std_num)              #计算sigma和delta
print('x,y,z的系统误差为：',sigma,'\nx,y,z的随机误差为：',delta)
cal_margin(sigma,delta);                                              #计算外放边界

In [ ]:
#保存excel代码段
writer = pd.ExcelWriter(r'E:\paper\摆位误差分析\test\test.xls')
final_data.to_excel(writer,sheet_name = 'Sheet1',index = False)
writer.save()
writer.close()

In [ ]:
final_data.plot.bar(y='x')


In [ ]:
final_data.boxplot()


In [ ]:
# 绘制多子图
final_data.hist(color="gray", bins=50)#, alpha=0.5


In [ ]:
data_head.boxplot()

In [ ]:
#摆位数据录入
error_path = r'E:\paper\摆位误差分析\test\摆位有效数据.xls'
error_data = pd.read_excel(error_path,names=None,sheet_name='Sheet1',index_col= None,header=0)
error_data.sort_values('ID', ascending=True,inplace = True)
error_data.set_index('ID')
error_data = error_data.loc[:,['ID','name','gender','age','position','x','y','z']]

In [ ]:
#登记数据录入
regs_path = r'E:\paper\摆位误差分析\test\登记数据.xlsx'
regs_data = pd.read_excel(regs_path,names=None,sheet_name='合并',index_col= None,header=None)
regs_data.drop([0],inplace = True)
regs_data.rename(columns ={0:'data',1:'ID',2:'name',3:'gender',4:'age',5:'part',
                              6:'disease',7:'doctor',8:'height',9:'weight'},inplace =True)

In [ ]:
#录入BMI数据
#regs_data.drop(['data','part','doctor'],axis = 1,inplace = True)   #删除指定列
bmi_info = regs_data[['ID','height','weight']]
bmi_info['ID'] = bmi_info.loc[:,'ID'].apply(pd.to_numeric, errors='coerce')#.fillna(999)
bmi_info.dropna(inplace = True)
bmi_info.sort_values('ID', ascending=True,inplace = True)
bmi_info

In [ ]:
error_bmi = error_data.merge(bmi_info, left_on='ID', right_on='ID', how='inner')
error_bmi.sort_values('ID', ascending=True,inplace = True)
#error_data.join(diseases)
error_bmi.info()

In [ ]:
#整合定位的ID号和诊断信息
sim2020_path = r'E:\paper\摆位误差分析\test\2020年定位记录表.xlsx'
sim_2020 = pd.read_excel(sim2020_path,names=None,sheet_name='定位',index_col= None,header=None)
sim_2020.drop([0],inplace = True)
sim_2020.rename(columns ={0:'data',1:'ID',2:'name',3:'gender',4:'age',5:'part',
                              6:'no.1',7:'no.2',8:'disease',9:'weight'},inplace =True)
t_2020 = sim_2020[['ID','disease']]
#t_2020.info()

sim2021_path = r'E:\paper\摆位误差分析\test\2021年定位记录表.xlsx'
sim_2021 = pd.read_excel(sim2021_path,names=None,sheet_name='定位',index_col= None,header=None)
sim_2021.drop([0],inplace = True)
sim_2021.rename(columns ={0:'data',1:'ID',2:'name',3:'gender',4:'age',5:'part',
                              6:'no.1',7:'no.2',8:'disease',9:'weight'},inplace =True)
t_2021 = sim_2021[['ID','disease']]
#t_2021.drop([0],inplace = True)
sim_all = pd.concat([t_2020,t_2021],axis=0)
#t_2021.info()

sim2022_path = r'E:\paper\摆位误差分析\test\2022年定位记录表.xlsx'
sim_2022 = pd.read_excel(sim2022_path,names=None,sheet_name='定位',index_col= None,header=None)
sim_2022.drop([0],inplace = True)
sim_2022.rename(columns ={0:'data',1:'ID',2:'name',3:'gender',4:'age',5:'part',
                              6:'no.1',7:'no.2',8:'disease',9:'weight'},inplace =True)
t_2022 = sim_2022[['ID','disease']]
#t_2022.drop([0],inplace = True)
#t_2022.info()
sim_all = pd.concat([sim_all,t_2022],axis=0)


In [ ]:
sim_all['ID'] = sim_all['ID'].apply(pd.to_numeric, errors='coerce')
sim_all.dropna(inplace = True)
sim_all.sort_values('ID', ascending=True,inplace = True)
sim_all

In [ ]:
error_bmi_sim = error_bmi.merge(sim_all, left_on='ID', right_on='ID', how='inner')
error_bmi_sim.drop_duplicates(inplace =True) 
error_bmi_sim

In [ ]:
error_bmi_sim.info()

In [ ]:
error_bmi_sim.drop_duplicates(['ID','name','x','y','z'],keep = 'first',inplace = True) 

In [ ]:
error_bmi_sim.info()

In [ ]:
#保存excel代码段
writer = pd.ExcelWriter(r'E:\paper\摆位误差分析\test\final_error_bmi_sim_final.xls')
error_bmi_sim.to_excel(writer,sheet_name = 'Sheet1',index = False)
writer.save()
writer.close()

In [ ]:
error_bmi_sim.groupby('ID').count()

In [ ]:
error_bmi_sim[['height','weight']] = error_bmi_sim[['height','weight']].apply(pd.to_numeric, errors='coerce') 
error_bmi_sim.eval('bmi = weight/(height/100)',inplace = True)#, inplace=True

In [ ]:
#保存excel代码段
writer = pd.ExcelWriter(r'E:\paper\摆位误差分析\test\final_error_bmi_sim_cal.xls')
error_bmi_sim.to_excel(writer,sheet_name = 'Sheet1',index = False)
writer.save()
writer.close()

In [ ]:
path = r'E:\paper\摆位误差分析\test\final_error_bmi_sim_cal.xls'
error_bmi_sim = pd.read_excel(path,names=None,sheet_name='Sheet1',index_col= None,header=0)
final_lung = error_bmi_sim.loc[error_bmi_sim['disease'].str.contains('肺')]

In [ ]:
final_lung.info()

In [ ]:
final_lung.dropna(inplace = True)
final_lung.info()

In [ ]:
#保存excel代码段
writer = pd.ExcelWriter(r'E:\paper\摆位误差分析\test\final_lung.xls')
final_lung.to_excel(writer,sheet_name = 'Sheet1',index = False)
writer.save()
writer.close()

In [ ]:
path = r'E:\paper\摆位误差分析\test\final_lung.xls'
final_lung = pd.read_excel(path,names=None,sheet_name='Sheet1',index_col= None,header=0)

In [ ]:
final_lung.eval('bmi = weight/((height/100)**2)',inplace = True)
final_lung

In [ ]:
final_lung.sort_values('ID', ascending=True,inplace = True)

In [ ]:
#保存excel代码段
writer = pd.ExcelWriter(r'E:\paper\摆位误差分析\test\final_lung.xls')
final_lung.to_excel(writer,sheet_name = 'Sheet1',index = False)
writer.save()
writer.close()

In [ ]:
#摆位登记病种和定位登记病种
final_lung = final_lung.loc[final_lung['disease'].str.contains('肺') & final_lung['position'].str.contains('肺|胸')]

In [ ]:
final_lung.info()

In [ ]:
writer = pd.ExcelWriter(r'E:\paper\摆位误差分析\test\final_lung.xls')
final_lung.to_excel(writer,sheet_name = 'Sheet1',index = False)
writer.save()
writer.close()

In [ ]:
final_lung.groupby('ID').count()

In [ ]:
mean_num,std_num = cal_mean_std(final_lung)    #计算不同患者的摆位误差的平均值和标准差
sigma,delta = cal_sigma_delta(mean_num,std_num)              #计算sigma和delta
cal_margin(sigma,delta);                                              #计算外放边界

In [ ]:
path = r'E:\paper\摆位误差分析\test\final_lung.xls'
final_lung = pd.read_excel(path,names=None,sheet_name='Sheet1',index_col= None,header=0)

In [ ]:
final_lung['age'] = final_lung['age'].apply(pd.to_numeric, errors='coerce')
age_list = final_lung.groupby('ID')['age'].mean().tolist()#['age']

In [ ]:
np.min(age_list)

In [ ]:
lung_male = final_lung.loc[(final_lung['gender'].str.contains('男'))]#.tolist(),'ID'
lung_female = final_lung.loc[(final_lung['gender'].str.contains('女'))]#.tolist(),'ID'

In [ ]:
#print(set(male_ID)&set(female_ID))  #转化成集合求交集
print(set(lung_male)&set(lung_female))

In [ ]:
lung_male.groupby('ID').count()

In [ ]:
lung_female.groupby('ID').count()